In [ ]:
from gerrychain import Graph

In [ ]:
# Read Oklahoma county graph from the json file "OK_county.json"
filepath = 'C:\\districting-data\\'
filename = 'OK_county.json'

# GerryChain has a built-in function for reading graphs of this type:
G = Graph.from_json( filepath + filename )

In [ ]:
# For each node, print the node #, county name, population, and lat-long coordinates
for node in G.nodes:
    name = G.nodes[node]["NAME10"]
    population = G.nodes[node]['TOTPOP']
    
    # query lat and long coordinates 
    
    
    print("Node",node,"is",name,"County, which has population",population,"and is centered at (")

In [ ]:
# pip install geopy

In [ ]:
# what is the "distance" between Payne County (node 0), Oklahoma County (node 15), and Tulsa County (node 23)?


# Store centroid location as ( long, lat )


# Print the distance in miles


In [ ]:
# create distance dictionary



In [ ]:
# check the dictionary by printing the Payne County -> Oklahoma County distance



In [ ]:
# Let's impose a 1% population deviation (+/- 0.5%)
deviation = 0.01

import math
k = 5          # number of districts
total_population = sum(G.nodes[node]['TOTPOP'] for node in G.nodes)

L = math.ceil((1-deviation/2)*total_population/k)
U = math.floor((1+deviation/2)*total_population/k)
print("Using L =",L,"and U =",U,"and k =",k)

In [ ]:
import gurobipy as gp
from gurobipy import GRB

# create model 


# create x[i,j] variable which equals one when county i 
#    is assigned to (the district centered at) county j


In [ ]:
# objective is to minimize the moment of inertia: sum (d^2 * p * x over all i and j)


In [ ]:
# add constraints saying that each county i is assigned to one district


# add constraint saying there should be k district centers


# add constraints that say: if j roots a district, then its population is between L and U.


# add coupling constraints saying that if i is assigned to j, then j is a center.




In [ ]:
# solve, making sure to set a 0.00% MIP gap tolerance(!)


m.optimize()

In [ ]:
# print the objective value


# retrieve the districts and their populations
#    but first get the district "centers"

districts = [ [i for i in G.nodes if x[i,j].x > 0.5] for j in centers]
district_counties = [ [ G.nodes[i]["NAME10"] for i in districts[j] ] for j in range(k)]
district_populations = [ sum(G.nodes[i]["TOTPOP"] for i in districts[j]) for j in range(k) ]

# print district info
for j in range(k):
    print("District",j,"has population",district_populations[j],"and contains counties",district_counties[j])

In [ ]:
# Let's draw it on a map
import geopandas as gpd

In [ ]:
# Read Oklahoma county shapefile from "OK_county.shp"
filepath = 'C:\\districting-data\\'
filename = 'OK_county.shp'

# Read geopandas dataframe from file
df = gpd.read_file( filepath + filename )

In [ ]:
# Which district is each county assigned to?
assignment = [ -1 for u in G.nodes ]
    
# for each district j
for j in range(len(districts)):
    
    # for each node i in this district
    for i in districts[j]:
        
        # What is its GEOID?
        geoID = G.nodes[i]["GEOID10"]
        
        # Need to find this GEOID in the dataframe
        for u in G.nodes:
            if geoID == df['GEOID10'][u]: # Found it
                assignment[u] = j # Node u from the dataframe should be assigned to district j

# Now add the assignments to a column of the dataframe and map it
df['assignment'] = assignment
my_fig = df.plot(column='assignment').get_figure()